In [1]:
import tkinter as tk
import tkinter.ttk as ttk
from collections import OrderedDict
import itertools
import copy
import json
from hello.hello3 import HelloXML
import hello.mock.util as hmu
import requests
import http.server
from urllib.parse import urlparse, parse_qs as ulib_parseqs


In [2]:
class ScrollFrame(tk.Frame):
    def __init__(self, master, height=100, width=100):
        super().__init__(master, relief=tk.GROOVE, width=width, height=height)
        self.width = width
        self.height = height
        self.canvas = tk.Canvas(self)
        self.inner_frame = tk.Frame(self.canvas, width=width)
        self.scrollbar = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.window = self.canvas.create_window((0,0), window=self.inner_frame, anchor='nw')
        self.inner_frame.bind("<Configure>", self._on_frame_configure)
        self.canvas.bind("<Configure>", self._on_canvas_configure)
        self.canvas.bind("<MouseWheel>", self.on_mouse_scroll)
        self.items = []
        
    def add_item(self, item):
        self.items.append(item)
    
    def create_widget(self, klass, *args, **kw):
        w = klass(self.inner_frame, *args, **kw)
        self.add_item(w)
        w.bind("<MouseWheel>", self.on_mouse_scroll)
        return w
    
    def _configure_widgets(self):
        self.canvas.pack(side="left", fill=tk.BOTH)
        self.scrollbar.pack(side="right", fill="y")
        self.canvas.configure(width=self.width-self.scrollbar.winfo_width())
        #self.inner_frame.pack_configure(fill=tk.X)
        for it in self.items:
            it.pack(anchor='w', expand=True, fill=tk.X)

    def pack(self, **kw):
        super().pack(**kw)
        self._configure_widgets()
    
    def grid(self, **kw):
        super().grid(**kw)
        self._configure_widgets()            
            
    def _on_frame_configure(self, e):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"),
                              width=self.width, height=self.height)
        
    def on_mouse_scroll(self, e):
        self.canvas.yview_scroll(-int(e.delta/120), "units")
        
    def _on_canvas_configure(self, e):
        self.canvas.itemconfig(self.window,
                               width=self._calc_window_width(e.width),)
                               #height=e.height)
    
    def _calc_window_width(self, width):
        # take 2 off the width to account for relief on buttons
        return width - self.scrollbar.winfo_width()-2
        
        

In [3]:
class TabBar(ScrollFrame):
    def add_tab(self, text, command):
        """ Register a button with the given text and callback """
        return self.create_widget(tk.Button, text=text, command=command)
    
class SimpleEntry(tk.Entry):
    def __init__(self, master, text="", **kw):
        tv = kw.pop('textvariable', None) or tk.StringVar(None, text)
        kw['textvariable'] = tv
        super().__init__(master, **kw)

In [4]:
class Item():
    def __init__(self, events, value, label, entry, indicator, typ=str, fmt="%s"):
        self.events = events
        self.label = label
        self.entry = entry
        self.indicator = indicator
        self.typ = typ
        self.fmt = fmt
        self.value = value
        self.indicator.config(text=self.format_value(value))
        self.bind("<Return>")
        self.bind("<FocusOut>")
        self.value_changed_event = "ITEM_VALUE_MODIFIED"
        self.event_key = None
        
    def bind(self, ev):
        self.entry.bind(ev, self.accept_number)
        
    def on_err(self, v):
        if self.event_key is not None:
            try:
                s = "%s %s" % self.event_key
            except:
                s = str(self.event_key)
        else:
            s = self.text
        print("Invalid value for '%s':" % s, repr(v))

    def set_event(self, ev, key):
        self.value_changed_event = ev
        self.event_key = key
        
    def accept_number(self, e):
        v = self.entry.get()
        if not v:
            return
        try:
            v2 = self.format_value(v)
        except (TypeError, ValueError):
            self.on_err(v)
            return
        self.indicator.config(text=v2)
        self.value = v
        self.events.fire(self.value_changed_event, self, e, self.event_key, v)
        
    def format_value(self, v):
        return self.fmt % self.typ(v)
    
    def set_label(self, v, format=True):
        if format:
            v = self.format_value(v)
        self.indicator.config(text=v)
        
    def get_value(self):
        return self.typ(self.value)
    
    def set_fmt(self, typ, fmt):
        self.typ = typ
        self.fmt = fmt
        self.indicator.config(text=self.format_value(self.value))

class ItemFrame():
    def __init__(self, master, events, text):
        self.master = master
        self.frame = ttk.LabelFrame(master, text=text)
        self.items = OrderedDict()
        self.text = text
        self.events = events
        
    def add_item(self, name, value, typ=str, fmt="%s"):
        label = tk.Label(self.frame, text=name, justify=tk.LEFT, anchor=tk.W)
        entry = SimpleEntry(self.frame)
        indicator = tk.Label(self.frame, justify=tk.LEFT, anchor=tk.W)
        self.items[name] = Item(self.events, value, label, entry, indicator, typ, fmt)
        return self.items[name]
        
    def grid(self, **kw):
        self.frame.grid(**kw)
        for i, item in enumerate(self.items.values()):
            item.label.grid(row=i, column=0)
            item.entry.grid(row=i, column=1)
            item.indicator.grid(row=i, column=2)
            item.label.config(width=12)
            item.entry.config(width=6)
            item.indicator.config(width=6)

In [26]:
class MockHTTPResponse():
    def __init__(self):
        self.status_code = 404
        self.content = ""
        self.headers = {"Content-Type": "text/plain"}
        
    @property 
    def content_type(self):
        return self.headers["Content-Type"]
    
    @content_type.setter
    def content_type(self, value):
        self.headers['Content-Type'] = value

        
def parse_qs(qs):
    dct = ulib_parseqs(qs)
    rv = {}
    for k, v in dct.items():
        v = v[0]
        if isinstance(v, str):
            v = v.lower()
        rv[k.lower()] = v
    return rv
        
        
class HelloHandler(http.server.SimpleHTTPRequestHandler):
    
    # Inherited attributes and methods
    close_connection = False
    timeout = 0

    def finish(self):
        pass
    
    def handle(self):
        while not self.close_connection:
            self.handle_one_request()
            
    def default_response(self):
        r = MockHTTPResponse()
        r.content = "Bad Request"
        r.status_code = 404
        self.http_response(r)
        
    def http_response(self, r):
        self.send_reply(r.status_code, r.content, r.content_type)
        
    def set_reply(self, code=200, content="", content_type="text/plain"):
        r = MockHTTPResponse()
        r.status_code = code
        r.content = content
        r.content_type = content_type
        self.response = r
        
    # Meat and potatoes
    def do_GET(self):
        """ Handle a get request. Parse request and fire event handlers for 
        server call or file path request events. This code is a bit spaghetti,
        because the server response needs to be synchronous. In order to 
        allow the application model objects to handle response processing
        cleanly, while keeping this class simple (and focused primarily on 
        server- related stuff), a stateful approach is used:
        
        1) `response` attribute is reset to 'None'
        2) Request is processed and events are fired with a reference 
        to this http object. 
        3) Event handlers will either:
          3.1) Perform a proxy request using the requests library
          3.2) Instantiate a MockHTTPResponse object with compatible interface 
               (only as much as this class requires)
        4) The Response will have status code, body, and content type set properly
        5) The Response will be assigned to this class's `response` attribute. 
        6) The Response will be processed and returned as the HTTP Response. 
        """
        self.response = None
        parsed = urlparse(self.path)
        if parsed.path == "/webservice/interface/":
            params = parse_qs(parsed.query)
            call = params.pop("call", None)
            call = call or params.pop("getfile", None)
            if not call:
                self.send_error(405, "No call parameter found")
            self.server.events.fire("SERVER_CALL_REQUEST", self, None, call, params)
        else:
            self.server.events.fire("FILE_PATH_REQUEST", self, None, parsed.path)
        if self.response is None:
            self.default_response()
        else:
            self.http_response(self.response)

    def send_file_reply(self, code, path):
        
        extension = os.path.splitext(path)[1]
        if extension == ".png":
            content_type = "image/png"
        elif extension == ".css":
            content_type = "text/css"
        elif extension == ".html":
            content_type = "text/html"
        elif extension == ".js":
            content_type = "application/x-javascript"
        elif extension == ".gif":
            content_type = "image/gif"
        elif extension == ".ico":
            content_type = "image/x-icon"
        elif extension == ".min.map":
            content_type = "application/json"
        else:
            self.send_error(501, "Invalid Path Request: '%s'" % path)
            return
        
        with open(path, 'rb') as f:
                body = f.read()
        self.send_reply(code, body, content_type)
        
    def send_reply(self, code, body, content_type):
        if not isinstance(body, bytes):
            body = body.encode("utf-8")
        self.send_response(code)
        for h, v in self.headers.items():
            self.send_header(h, v)
        self.send_header("Content-Length", len(body))
        self.send_header("Content-Type", content_type)
        self.end_headers()
        self.wfile.write(body)
        self.wfile.flush()
        
    def send_error(self, code, msg, content_type=None):
        if isinstance(msg, str):
            msg = msg.encode("utf-8")
        self.send_reply(code, msg, "application/json")
    
class HelloServer(http.server.HTTPServer):
    timeout=0
    def __init__(self, events, host="localhost", port=12345):
        super().__init__((host, port), HelloHandler)
        self.events = events
        
    def process_request(self, request, client_address):
        self.finish_request(request, client_address)

In [27]:
_mv_json = """{"agitation":{"pv":0,"sp":30,"man":20,"mode":2,"error":0,"interlocked":0,"output":0},"temperature":{"pv":21.674983978271484,"sp":37,"man":0,"mode":0,"error":0,"interlocked":0,"output":100},"do":{"pv":107.49867248535156,"sp":50,"manUp":0,"manDown":50,"mode":2,"error":0,"outputUp":0,"outputDown":0},"ph":{"pv":14,"sp":7,"manUp":50,"manDown":7,"mode":2,"error":200,"outputUp":0,"outputDown":0},"pressure":{"pv":-0.00034421682357788086,"error":0},"level":{"pv":0,"error":0},"condenser":{"pv":25.133401870727539,"sp":35,"man":0,"mode":0,"error":0,"output":19.733196258544922},"maingas":{"pv":46.17816162109375,"man":5,"mode":1,"error":0,"interlocked":0},"MFCs":{"air":30.75819206237793,"co2":7.6610736846923828,"n2":7.7588958740234375,"o2":7.6193175315856934}}"""
_static_json = """{"network":{"local":false},"versions":{"rio":"V3.0.64","server":"V3.0.71","type":"Mag","size":15,"database":"V2.4","serialNumber":"01961CEE","temperatureInputs":2,"doInputs":2,"phInputs":2},"agitation":{"pvUnit":"RPM","manUnit":"%","manName":"Percent Power","pvDecimals":0,"manDecimals":1},"temperature":{"pvUnit":"°C","manUnit":"%","manName":"Main Heater Duty","pvDecimals":1,"manDecimals":1},"do":{"pvUnit":"%","manUpUnit":"%","manDownUnit":"%","manUpName":"O2","manDownName":"N2","pvDecimals":1,"manUpDecimals":1,"manDownDecimals":1},"ph":{"pvUnit":"","manUpUnit":"%","manDownUnit":"%","manUpName":"Base","manDownName":"CO2","pvDecimals":2,"manUpDecimals":1,"manDownDecimals":1},"pressure":{"pvUnit":"psi","pvDecimals":1},"level":{"pvUnit":"L","pvDecimals":1,"enableLevel":1},"condenser":{"pvUnit":"°C","manName":"Filter Oven Heater Duty","manUnit":"%","pvDecimals":1,"manDecimals":1},"maingas":{"pvUnit":"L/min","manName":"Total Flow","manUnit":"L/min","pvDecimals":2,"manDecimals":2},"mfcs":{"airPvDecimals":2,"co2PvDecimals":2,"n2PvDecimals":2,"o2PvDecimals":2}}"""
_doravalues_xml = """<?xml version="1.0" encoding="windows-1252" standalone="no" ?><Reply><Result>True</Result><Message><Cluster><Name></Name><NumElts>5</NumElts><String><Name>Machine Name</Name><Val>PBS3RnD2</Val></String><String><Name>Start Time</Name><Val>--</Val></String><String><Name>Elapsed</Name><Val>--</Val></String><String><Name>Batch</Name><Val>--</Val></String><String><Name>Sequence</Name><Val>Idle</Val></String></Cluster></Message></Reply>"""
_getconfig_xml = '<?xml version="1.0" encoding="windows-1252" standalone="no" ?><Reply><Result>True</Result><Message><Cluster><Name>System_Variables</Name><NumElts>12</NumElts><Cluster><Name>Temperature</Name><NumElts>8</NumElts><SGL><Name>P_Gain_(%/C)</Name><Val>25.00000</Val></SGL><SGL><Name>I_Time_(min)</Name><Val>5.00000</Val></SGL><SGL><Name>D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>Heat_Manual_Max_(%)</Name><Val>50.00000</Val></SGL><SGL><Name>Heat_Auto_Max_(%)</Name><Val>50.00000</Val></SGL><SGL><Name>Valid_High_(C)</Name><Val>110.00000</Val></SGL><SGL><Name>Valid_Low_(C)</Name><Val>-5.00000</Val></SGL><SGL><Name>Max_Delta_(C)</Name><Val>2.00000</Val></SGL></Cluster><Cluster><Name>Filter_Oven</Name><NumElts>5</NumElts><SGL><Name>P_Gain_(%/C)</Name><Val>1.50000</Val></SGL><SGL><Name>I_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>Heat_Manual_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>Heat_Auto_Max_(%)</Name><Val>100.00000</Val></SGL></Cluster><Cluster><Name>Agitation</Name><NumElts>13</NumElts><SGL><Name>P_Gain_(%/RPM)</Name><Val>0.10000</Val></SGL><SGL><Name>I_Time_(min)</Name><Val>4.00000E-3</Val></SGL><SGL><Name>D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>Minimum_(RPM)</Name><Val>3.00000</Val></SGL><SGL><Name>Pulse_Mode_Timeout_(s)</Name><Val>1.00000</Val></SGL><SGL><Name>Lookup_Mode_Timeout_(s)</Name><Val>60.00000</Val></SGL><SGL><Name>Lookup_Factor_(%/RPM)</Name><Val>1.79900</Val></SGL><SGL><Name>Power_Auto_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>Power_Auto_Min_(%)</Name><Val>3.50000</Val></SGL><SGL><Name>Auto_Max_Startup_(%)</Name><Val>10.00000</Val></SGL><SGL><Name>Power_Manual_Max_(%)</Name><Val>100.00000</Val></SGL><U8><Name>Number_of_Magnets</Name><Val>2</Val></U8><I32><Name>Samples_To_Average</Name><Val>3</Val></I32></Cluster><Cluster><Name>pH</Name><NumElts>18</NumElts><SGL><Name>Max_Delta</Name><Val>2.10000</Val></SGL><SGL><Name>Fail_Rate_(pH/min)</Name><Val>1.00000</Val></SGL><SGL><Name>CO2_P_Gain_(%/pH)</Name><Val>-200.00000</Val></SGL><SGL><Name>CO2_I_Time_(min)</Name><Val>2.00000</Val></SGL><SGL><Name>CO2_D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>CO2_Manual_Max_(%)</Name><Val>25.00000</Val></SGL><SGL><Name>CO2_Auto_Max_(%)</Name><Val>30.00000</Val></SGL><SGL><Name>Base_P_Gain_(%/pH)</Name><Val>10.00000</Val></SGL><SGL><Name>Base_I_Time_(min)</Name><Val>10.00000</Val></SGL><SGL><Name>Base_D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>Base_Manual_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>Base_Auto_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>Base_Wait_Time_(s)</Name><Val>60.00000</Val></SGL><U8><Name>A_Use_Temp_Comp?</Name><Val>1</Val></U8><U8><Name>B_Use_Temp_Comp?</Name><Val>1</Val></U8><SGL><Name>Deadband</Name><Val>2.00000E-2</Val></SGL><SGL><Name>Valid_High_(pH)</Name><Val>14.00000</Val></SGL><SGL><Name>Valid_Low_(pH)</Name><Val>0.00000</Val></SGL></Cluster><Cluster><Name>DO</Name><NumElts>14</NumElts><SGL><Name>Max_Delta</Name><Val>3.00000</Val></SGL><SGL><Name>Valid_High_(DO%)</Name><Val>200.00000</Val></SGL><SGL><Name>Valid_Low_(DO%)</Name><Val>-10.00000</Val></SGL><SGL><Name>O2_P_Gain_(%/DO%)</Name><Val>1.00000</Val></SGL><SGL><Name>O2_I_Time_(min)</Name><Val>10.00000</Val></SGL><SGL><Name>O2_D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>O2_Manual_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>O2_Auto_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>N2_P_Gain_(%/DO%)</Name><Val>-5.00000</Val></SGL><SGL><Name>N2_I_Time_(min)</Name><Val>10.00000</Val></SGL><SGL><Name>N2_D_Time_(min)</Name><Val>0.00000</Val></SGL><SGL><Name>N2_Manual_Max_(%)</Name><Val>100.00000</Val></SGL><SGL><Name>N2_Auto_Max_(%)</Name><Val>90.00000</Val></SGL><SGL><Name>Deadband_(DO%)</Name><Val>1.00000</Val></SGL></Cluster><Cluster><Name>Level</Name><NumElts>11</NumElts><SGL><Name>Radius_(cm)</Name><Val>13.75000</Val></SGL><SGL><Name>Empty_Level_(V)</Name><Val>-1.00000</Val></SGL><SGL><Name>Empty_Level_(L)</Name><Val>0.00000</Val></SGL><SGL><Name>cm/psi</Name><Val>70.35800</Val></SGL><SGL><Name>Vessel_Depth_(cm)</Name><Val>15.30000</Val></SGL><SGL><Name>Bottom_Gap_(cm)</Name><Val>5.00000</Val></SGL><U8><Name>Enable_Sensor_(0_or_1)</Name><Val>1</Val></U8><SGL><Name>CalLevelSlopeMax(psi/V)</Name><Val>10000.00000</Val></SGL><SGL><Name>CalLevelSlopeMin(psi/V)</Name><Val>-10000.00000</Val></SGL><SGL><Name>CalLevelInterceptMax(psi)</Name><Val>10000.00000</Val></SGL><SGL><Name>CalLevelInterceptMin(psi)</Name><Val>-10000.00000</Val></SGL></Cluster><Cluster><Name>Pressure</Name><NumElts>6</NumElts><SGL><Name>Disconnected_Pressure_(V)</Name><Val>5.00000E-3</Val></SGL><SGL><Name>CalPressureInterceptMax(psi)</Name><Val>1.50000</Val></SGL><SGL><Name>CalPressureInterceptMin(psi)</Name><Val>-1.50000</Val></SGL><SGL><Name>CalPressureSlopeMax(psi/V)</Name><Val>250.00000</Val></SGL><SGL><Name>CalPressureSlopeMin(psi/V)</Name><Val>150.00000</Val></SGL><U8><Name>Reusable_Sensor_(0_or_1)</Name><Val>0</Val></U8></Cluster><Cluster><Name>Gas_Data</Name><NumElts>13</NumElts><SGL><Name>CO2_Min_(LPM)</Name><Val>3.00000E-2</Val></SGL><SGL><Name>CO2_Off_(V)</Name><Val>0.00000</Val></SGL><SGL><Name>N2_Min_(LPM)</Name><Val>8.00000E-2</Val></SGL><SGL><Name>N2_Off_(V)</Name><Val>0.00000</Val></SGL><SGL><Name>Air_Min_(LPM)</Name><Val>8.00000E-2</Val></SGL><SGL><Name>Air_Off_(V)</Name><Val>-7.00000E-3</Val></SGL><SGL><Name>O2_Min_(LPM)</Name><Val>2.00000E-2</Val></SGL><SGL><Name>O2_Off_(V)</Name><Val>0.00000</Val></SGL><SGL><Name>PWM_On_Time_(s)</Name><Val>3.00000</Val></SGL><SGL><Name>PWM_Max_Period_(s)</Name><Val>60.00000</Val></SGL><SGL><Name>Mismatch_Thresh_(V)</Name><Val>0.10000</Val></SGL><SGL><Name>O2_Min_Volume_(L)</Name><Val>1.00000E-2</Val></SGL><SGL><Name>Manual_Max_(LPM)</Name><Val>2.00000</Val></SGL></Cluster><Cluster><Name>Safety</Name><NumElts>8</NumElts><SGL><Name>Min_Ag_Power_(%)</Name><Val>10.00000</Val></SGL><SGL><Name>Max_Temp_(C)</Name><Val>45.00000</Val></SGL><SGL><Name>Max_Pressure_(psi)</Name><Val>0.50000</Val></SGL><SGL><Name>Max_Pressure_Door_(psi)</Name><Val>0.20000</Val></SGL><SGL><Name>Min_Level_(L)</Name><Val>4.00000</Val></SGL><SGL><Name>Max_Level_(L)</Name><Val>17.00000</Val></SGL><SGL><Name>Buzzer_Period_(ms)</Name><Val>100.00000</Val></SGL><U8><Name>DoorPressureSensor_(0_or_1)</Name><Val>1</Val></U8></Cluster><Cluster><Name>Pumps</Name><NumElts>6</NumElts><U16><Name>Aux_Low_Duty</Name><Val>20000</Val></U16><U16><Name>Aux_Med_Duty</Name><Val>30000</Val></U16><SGL><Name>Base_On_Time_(s)</Name><Val>0.10000</Val></SGL><SGL><Name>Base_Max_Period_(s)</Name><Val>240.00000</Val></SGL><SGL><Name>Analog_Base_Speed_(RPM)</Name><Val>50.00000</Val></SGL><U8><Name>Sample_Reverse_CW_and_CCW_(0_or_1)</Name><Val>0</Val></U8></Cluster><Cluster><Name>Process_Alarms</Name><NumElts>32</NumElts><SGL><Name>Agitation_Low_Low_(RPM)</Name><Val>0.00000</Val></SGL><SGL><Name>Agitation_Low_(RPM)</Name><Val>0.00000</Val></SGL><SGL><Name>Agitation_High_(RPM)</Name><Val>1.00000E-3</Val></SGL><SGL><Name>Agitation_High_High_(RPM)</Name><Val>2.00000E-3</Val></SGL><SGL><Name>Temp_Low_Low_(C)</Name><Val>19.20000</Val></SGL><SGL><Name>Temp_Low_(C)</Name><Val>19.30000</Val></SGL><SGL><Name>Temp_High_(C)</Name><Val>21.00000</Val></SGL><SGL><Name>Temp_High_High_(C)</Name><Val>23.00000</Val></SGL><SGL><Name>DO_Low_Low_(%)</Name><Val>6.25000</Val></SGL><SGL><Name>DO_Low_(%)</Name><Val>7.00000</Val></SGL><SGL><Name>DO_High_(%)</Name><Val>24.00000</Val></SGL><SGL><Name>DO_High_High_(%)</Name><Val>25.00000</Val></SGL><SGL><Name>pH_Low_Low</Name><Val>1.50000</Val></SGL><SGL><Name>pH_Low</Name><Val>1.95000</Val></SGL><SGL><Name>pH_High</Name><Val>4.00000</Val></SGL><SGL><Name>pH_High_High</Name><Val>5.00000</Val></SGL><SGL><Name>Pressure_Low_Low_(psi)</Name><Val>-0.20000</Val></SGL><SGL><Name>Pressure_Low_(psi)</Name><Val>-0.10000</Val></SGL><SGL><Name>Pressure_High_(psi)</Name><Val>0.10000</Val></SGL><SGL><Name>Pressure_High_High_(psi)</Name><Val>0.40000</Val></SGL><SGL><Name>Level_Low_Low_(L)</Name><Val>1.00000</Val></SGL><SGL><Name>Level_Low_(L)</Name><Val>1.00000</Val></SGL><SGL><Name>Level_High_(L)</Name><Val>9.70000</Val></SGL><SGL><Name>Level_High_High_(L)</Name><Val>43.75000</Val></SGL><SGL><Name>Filter_Oven_Low_Low_(C)</Name><Val>0.00000</Val></SGL><SGL><Name>Filter_Oven_Low_(C)</Name><Val>20.00000</Val></SGL><SGL><Name>Filter_Oven_High_(C)</Name><Val>22.00000</Val></SGL><SGL><Name>Filter_Oven_High_High_(C)</Name><Val>26.20000</Val></SGL><SGL><Name>Main_Gas_Low_Low_(LPM)</Name><Val>0.00000</Val></SGL><SGL><Name>Main_Gas_Low_(LPM)</Name><Val>1.00000E-3</Val></SGL><SGL><Name>Main_Gas_High_(LPM)</Name><Val>1.60000</Val></SGL><SGL><Name>Main_Gas_High_High_(LPM)</Name><Val>47.50000</Val></SGL></Cluster><Cluster><Name>System</Name><NumElts>2</NumElts><SGL><Name>Max_Data_Log_Interval_(min)</Name><Val>60.00000</Val></SGL><SGL><Name>Alarm_Snooze_Time_(s)</Name><Val>300.00000</Val></SGL></Cluster></Cluster></Message></Reply>'
STICKY_ALL = (tk.N, tk.S, tk.E, tk.W)

class FrameWidget1(tk.Frame):
    ITEM_EVENT = "ITEM_VALUE_MODIFIED"
    CALL_NAME = "DEFAULT"
    def __init__(self, master, events, **kw):
        super().__init__(master, **kw)
        self.events = events
        self.events.create_event(self.ITEM_EVENT)
        self.events.register(self.ITEM_EVENT, self.on_item_value_modified)
        self.init_data()
        self.frames = OrderedDict()
        self.setup_frames()
        self.configure_frames()
            
    def configure_frames(self):
        pass
    
    def init_data(self):
        self.data = {}
        
    def sync(self):
        for k in self.data:
            f = self.frames[k]
            for k2 in self.data[k2]:
                it = f.items[k2]
                self.data[k][k2] = it.get_value()
                
    def setup_frames(self):
        max_rows = max(len(c) for c in self.data)
        
        for i, k1 in enumerate(sorted(self.data)):
            r = i // 3
            c = i % 3
            f = ItemFrame(self, self.events, k1)
            for i2, k2 in enumerate(self.data[k1]):
                it = f.add_item(k2, self.data[k1][k2])
                item_key = (k1, k2)
                it.set_event(self.ITEM_EVENT, item_key)
            self.frames[k1] = f
            f.grid(rowspan=max_rows, column=c, row=r*max_rows, sticky=STICKY_ALL)
            
    def jsonify(self):
        return json.dumps(self.data, indent=4)
    
    def xmlify(self):
        return hmu(self.data, "Message")
    
    def on_item_value_modified(self, event, item, e, key, value):
        k1, k2 = key
        c = self.data[k1]
        c[k2] = value
    
    def on_err(self, v):
        print("Invalid value for '%s %s':" % (self.text, name), repr(v))
        
    def items(self):
        for frame in self.frames.values():
            yield from frame.items.values()

class GetMainValuesWidget(FrameWidget1):
    ITEM_EVENT = "MAINVALUES_ITEM_VALUE"
    CALL_NAME = "getMainValues"
            
    def init_data(self):
        self.data = json.loads(_mv_json)
            
    def configure_frames(self):
        for it in self.items():
            it.set_fmt(float, "%.2f")
    
class GetStaticWidget(FrameWidget1):
    ITEM_EVENT = "GETSTATIC_ITEM_VALUE"
    CALL_NAME = "getStatic"
                
    def init_data(self):
        self.data = json.loads(_static_json)
                
    def configure_frames(self):
        for it in self.items():
            it.label.config(width=15)
            it.indicator.config(width=15)
        
        
class FrameWidget2(tk.Frame):
    def setup_frame(self):
        for field in self.data:
            self.iframe.add_item(field, self.data[field])
        self.iframe.grid(sticky=STICKY_ALL)
        
    
class GetDoraValuesWidget(FrameWidget1):
    ITEM_EVENT = "DORA_ITEM_VALUE"
    CALL_NAME = "getDORAValues"
                
    def configure_frames(self):
        for it in self.items():
            it.indicator.config(width=15)
            
    def init_data(self):
        data = HelloXML(_doravalues_xml).data[None]
        self.data = {"getDORAValues": data}
                
    def xmlify(self):
        d = self.data['getDORAValues']
        xml = """
<?xml version="1.0" encoding="windows-1252" standalone="no" ?>
<Reply>
<Result>True</Result>
<Message><Cluster>
<Name></Name>
<NumElts>5</NumElts>
<String>
<Name>Machine Name</Name>
<Val>%s</Val>
</String>
<String>
<Name>Start Time</Name>
<Val>%s</Val>
</String>
<String>
<Name>Elapsed</Name>
<Val>%s</Val>
</String>
<String>
<Name>Batch</Name>
<Val>%s</Val>
</String>
<String>
<Name>Sequence</Name>
<Val>%s</Val>
</String>
</Cluster>
</Message>
</Reply>""" % (d["Machine Name"], d["Start Time"], 
               d["Elapsed"], d["Batch"], d["Sequence"])
                
            
class GetConfigWidget(FrameWidget1):
    ITEM_EVENT = "CONFIG_ITEM_VALUE"
                
    def init_data(self):
        self.data = HelloXML(_getconfig_xml).data['System_Variables']
        
    def configure_frames(self):
        for it in self.items():
            it.label.config(width=20)
            it.indicator.config(width=6)

In [28]:
_server_calls_ = [
    "getMainValues",
    "getStatic",
    "getDORAValues",
    "getTrendData",
    "set",
    "clearAlarm",
    "clearAlarmsbyType",
    "clearAllAlarms",
    "getConfig"
]

_server_call_widgets_ = {
    "getMainValues": GetMainValuesWidget,
    "getStatic": GetStaticWidget,
    "getDORAValues": GetDoraValuesWidget,
    "getTrendData": GetMainValuesWidget,
    "set": GetMainValuesWidget,
    "clearAlarm": GetMainValuesWidget,
    "clearAlarmsbyType": GetMainValuesWidget,
    "clearAllAlarms": GetMainValuesWidget,
    "getConfig": GetConfigWidget
}

def create_server_call_frame(call, master, events, **kw):
    return _server_call_widgets_[call](master, events, **kw)

In [29]:
_unregistered = []
def _dbg_events_(events, ev):
    if ev not in events.handlers:
        _unregistered.append(ev)
        print("Unregistered event:", ev)

class EventDispatcher():
    def __init__(self):
        self.handlers = {}
        
    def create_event(self, ev):
        self.handlers[ev] = self.handlers.get(ev, [])
        
    def register(self, ev, handler):
        if ev not in self.handlers:
            self.create_event(ev)
        self.handlers[ev].append(handler)
        
    def unregister(self, ev, handler):
        self.handlers[ev].remove(handler)
        
    def fire(self, ev, ob, e, *args):
        _dbg_events_(self, ev)
        for h in self.handlers.get(ev, ()):
            h(ev, ob, e, *args)
            

In [30]:
class SimulatorWindow():
    def __init__(self, events):
        self.events = events
        self.events.create_event("IP_ADDRESS_MODIFIED")
        self.events.create_event("MENU_ITEM_CLICKED")
        self.height = 750
        self.width = 900
        self.root = tk.Tk()
        self.display_frame = ScrollFrame(self.root, self.height*0.8, self.width*0.8)
        self.root.geometry("%dx%d"%(self.width, self.height))
        self.tablabel = ttk.Label(self.root, text="Available Server Calls:")
        self.tabbar = TabBar(self.root)
        self.tabs = {}
        self.frames = {}
        self.current = None
        
        self.ip_frame = ttk.LabelFrame(self.root, text="Proxy:")
        
        self.ip_label = tk.Label(self.ip_frame, justify=tk.LEFT, anchor=tk.W)
        self.ip_entry = tk.Entry(self.ip_frame)
        self.ip_entry.grid(column=0, row=0)
        self.ip_label.grid(column=0, row=1)
        self.ip_entry.bind("<Return>", self.IP_ADDRESS_MODIFIED)
        self.ip_entry.bind("<FocusOut>", self.IP_ADDRESS_MODIFIED)
        self.ip_label.config(width=15)  
        #self.create_tabs()
        
    def IP_ADDRESS_MODIFIED(self, e):
        self.set_ip(self.ip_entry.get())
        
    def set_ip(self, ip, e=None):
        self.ip_label.config(text=ip)
        self.events.fire("IP_ADDRESS_MODIFIED", self, e, ip)
        
    def create_tabs(self):
        
        def _on_cmd(call):
            def on_cmd():
                self.on_menu_item_clicked(call)
            return on_cmd
        
        for call in _server_calls_:
            tab = self.tabbar.add_tab(call, _on_cmd(call))
            frame = create_server_call_frame(call, self.display_frame.inner_frame, self.events)
            frame.bind("<MouseWheel>", self.display_frame.on_mouse_scroll)
            self.tabs[call] = tab
            self.frames[call] = frame
            
    def add_tab(self, call, Widget, **kw):
        def _on_cmd(call):
            def on_cmd():
                self.on_menu_item_clicked(call)
            return on_cmd
        w = Widget(self.display_frame.inner_frame, self.events, **kw)
        tab = self.tabbar.add_tab(call, _on_cmd(call))
        w.bind("<MouseWheel>", self.display_frame.on_mouse_scroll)
        self.tabs[call] = tab
        self.frames[call] = w
            
    def on_menu_item_clicked(self, item):
        if item == self.current:
            return

        if self.current is not None:
            self.frames[self.current].pack_forget()
            self.tabs[self.current].config(relief=tk.RAISED)

        self.frames[item].pack(side="left", fill=tk.BOTH)
        self.tabs[item].config(relief=tk.FLAT)
        
        self.current = item
        self.events.fire("MENU_ITEM_CLICKED", self, None, item)
        
    def current_frame(self):
        return self.frames[self.current]
        
    def mainloop(self):
        self.show()
        self.root.mainloop()
        
    def show(self):
        self.tablabel.pack(side="top", anchor='nw')
        self.tabbar.pack(side="left", fill='y')
        self.ip_frame.pack(side="top", anchor="ne")
        self.display_frame.pack(side="left", fill="y")
        self.tabs["getMainValues"].invoke()



In [31]:
class ProxyConnection():
    def __init__(self, base_path):
        self.sess = requests.Session()
        self.base_path = base_path
        
    def relay(self, request):
        headers = self.request.headers
        cmd = self.request.command
        path = self.request.path
        return self.sess.request(cmd, self.base_path+path, None, headers)
    
    def close(self):
        self.sess.close()
        

In [32]:
import threading
import http.client

def _urlify(ip):
    return "https://" + ip

class HelloSimulator():
    def __init__(self, proxy_ip="71.189.82.196:84"):
        self.events = EventDispatcher()
        self.window = SimulatorWindow(self.events)
        
        self.proxy = None
        self.events.register("IP_ADDRESS_MODIFIED", self.new_ip)
        self.window.set_ip(proxy_ip)
        
        self.server = HelloServer(self.events, "localhost", 12345)
        self.server_running = True
        self.server_thread = threading.Thread(None, self.run_server, daemon=True)
        self.server_thread.start()
        self.events.register("SERVER_CALL_REQUEST", self.on_server_call_request)
        
        self.call_frames = {}
        self.create_tabs()
        
    def on_server_call_request(self, ev, handler, _, call, params):
        frame = self.call_frames.get(call)
        if frame is None:
            handler.set_reply(405, "Invalid call: %r" % call)
        json = params.get("json", False)
        if json:
            content = frame.jsonify()
            ct = "application/json"
        else:
            content = frame.xmlify()
            ct = "application/xml"
        handler.set_reply(200, content, ct)            
        
    def run_server(self):
        while self.server_running:
            self.server.handle_request()
            
    def create_tabs(self):       
        for call, W in _server_call_widgets_.items():
            self.window.add_tab(call, W)
            
            # for case insensitivity in url processing
            self.call_frames[call.lower()] = self.window.frames[call]
        
    def new_ip(self, ev, window, e, ip):
        if self.proxy is not None:
            self.proxy.close()
        self.proxy = ProxyConnection(_urlify(ip))
    
    def run(self):
        self.window.mainloop()
        self.server_running = False

In [33]:
import gc
gc.collect()
HelloSimulator().run()

ON_SERVER_CALL_REQUEST SERVER_CALL_REQUEST <__main__.HelloHandler object at 0x0000023DEE8BFC88> getmainvalues {'json': '1'}


127.0.0.1 - - [21/Dec/2016 17:29:08] "GET /webservice/interface/?&call=getmainvalues&json=1 HTTP/1.1" 200 -


In [ ]:
sticky_all = (tk.N, tk.E, tk.W, tk.S)

class Item():
    def __init__(self, master, name, value=0, isnumber=True, decimals=2):
        self.name = name
        #self.frame = tk.Frame(master)
        self.label = ttk.Label(master, text=name, width=12)
        self.textvar = tk.StringVar()
        self.entry = ttk.Entry(master, textvariable=self.textvar)
        self.report = ttk.Label(master)
        self.isnumber = isnumber
        self.fmt = "%%.%df" % decimals
        self.textvar.set(self.format_value(value))
        self.value = value
        self.update_report(value)
        
        if self.isnumber:
            self.entry.bind("<Return>", self.accept_number)
        else:
            raise NotImplementedError        
        
    def accept_number(self, e):
        val = self.input_val()
        try:
            float(val)
        except ValueError:
            self.entry.bell()
            self.textvar.set(self.value)
        else:
            self.update_value(val)
        
    def format_value(self, value):
        return self.fmt % float(value)
        
    def update_value(self, value):
        self.update_report(value)
        self.value = value

    def input_val(self):
        return self.textvar.get()
    
    def grid(self, row=1, col=1):
        #self.frame.grid(row=row, column=col)
        self.label.grid(column=1, row=row, sticky=(tk.E,))
        self.entry.grid(column=2, row=row, sticky=(tk.E, tk.W))
        self.report.grid(column=3, row=row, sticky=(tk.E,))
        return 1
        
    def update_report(self, val):
        val = self.format_value(val)
        self.report.config(text=val)

class ItemList():
    def __init__(self, master, name, fields):
        self.frame = tk.LabelFrame(master, text=name)
        self.items = OrderedDict()
        self.name = name
        for f, val in fields.items():
            self.items[f] = Item(self.frame, f, val)
            
    def grid(self, row=1, col=1, cur_row=0):
        self.frame.grid(row=row, column=col, sticky=sticky_all)
        nrows = 0
        for i, item in enumerate(self.items.values()):
            nrows += item.grid(i+cur_row, col*3)
        return nrows
    
    def iteritems(self):
        return list(self.items.values())
    
    def dictify(self):
        v = {k: item.value for k, item in self.items.items()}
        return v
            
class MainValuesFrame():
    def __init__(self, master, mv):
        self.groups = OrderedDict()
        self.frame = ttk.LabelFrame(master, text="HELLO!")
        for group in mv:
            self.groups[group] = ItemList(self.frame, group, mv[group])
        self.apply_btn = ttk.Button(self.frame, text="Apply", command=self.apply)
        self.master = master
            
    def apply(self):
        for item in self.iteritems():
            value = item.input_val()
            item.update_value(value)
    
    def grid(self):
        subcols = 3
        nrows = 0
        for i, frame in enumerate(self.groups.values()):
            nrows += frame.grid(i//subcols, i%subcols, nrows)   
        self.frame.grid(columnspan=subcols, rowspan=nrows)
        #self.frame.grid()
        self.apply_btn.grid(column=(subcols-1)//2, row=nrows+1)
        for item in self.iteritems():
            item.label.config(width=12)
            item.entry.config(width=6)
            item.report.config(width=6)
                            
    def iteritems(self):
        items = []
        for frame in self.groups.values():
             items.extend(frame.iteritems())
        return items
    
    def dictify(self):
        rv = {k: group.dictify() for k, group in self.groups.items()}
        return rv

            
        

In [ ]:
import threading
import queue

class RepeatedTask():
    def __init__(self, master, func, interval):
        self.master = master
        self.func = func
        self.interval = interval
        self.after_id = None
        
    def schedule(self):
        self.after_id = self.master.after(self.interval, self.func)
    
    def cancel(self):
        if self.after_id:
            self.master.after_cancel(self.after_id)
        
    def update(self):
        try:
            self.func()
        finally:
            self.schedule()

class Window():
    def __init__(self, server=None, update_interval=0.5):
        self.tk = tk.Tk()
        self.server = server
        self.update_interval=0.5
        self.frames = OrderedDict()
        self.tasks = []
        self.server_thread = None
        self.server_running = True
        
    def set_server(self, server):
        self.server = server
        
    def mainloop(self):
        if not self.server:
            raise AttributeError("Server not set!")
        self.server_thread = threading.Thread(None, self.run_server, daemon=True)
        self.server_thread.start()
        self.tk.mainloop()
        self.server_running = False
        self.server_thread.join()
    
    def run_server(self):
        while self.server_running:
            print("Getting request")
            try:
                request, client_address = self.server.get_request()
            except OSError as e:
                print(e)
                continue
            if self.server.verify_request(request, client_address):
                try:
                    print("processing request")
                    self.server.process_request(request, client_address)
                except:
                    self.server.handle_error(request, client_address)
                    self.server.shutdown_request(request)
            else:
                self.server.shutdown_request(request)
    

In [ ]:
import socket
import ssl
import http.server
from urllib.parse import urlparse as url_parse, parse_qs as ulib_parseqs
import os

usr = os.path.expanduser("~")
serve_path = os.path.join(usr, "Personal", "test")
SERVER_DIR = os.path.abspath(serve_path)
os.makedirs(SERVER_DIR, exist_ok=True)

class Server():
    def __init__(self, host="localhost", port=12345, listen=1):
        self.connections = []
        self.sock = self.init_sock(host, port)
        self.listen = listen
        self.configure_socket(listen)
    
    def configure_socket(self, listen):
        self.sock.listen(listen)
        
    def init_sock(self, host, port):
        s = socket.socket()
        s.bind((host, port))
        return s
    
def parse_qs(qs):
    dct = ulib_parseqs(qs)
    rv = {}
    for k, v in dct.items():
        v = v[0]
        if isinstance(v, str):
            v = v.lower()
        rv[k.lower()] = v
    return rv
    

class BadServerCall(Exception):
    def __init__(self, code=405, msg="", content_type="application/json"):
        self.code = code
        self.args = msg,
        if not isinstance(msg, bytes):
            msg = msg.encode('utf-8')
        self.msg = msg
        self.content_type = content_type
        
        
class ServerBackend():
    def __init__(self, mv_frame):
        self.mv_frame = mv_frame
        self.calls = {
            "getmainvalues": self.do_getMainvalues,
        }
    def do_getMainvalues(self, args):
        
        mv = self.mv_frame.dictify()
        mv = json.dumps(mv)
        
        # Real web server behavior...
        if "json" not in args:
            mv = "<Reply><Result>True</Result><Message>%s</Message></Reply>" % mv
            content_type = "application/xml"
        else:
            content_type = "application/json"
        return mv.encode("utf-8"), content_type

    def execute(self, call, params):
        handler = self.calls[call]
        return handler(params)
    
    
class HelloHandler(http.server.SimpleHTTPRequestHandler):
    
    # Inherited attributes and methods
    close_connection = False
    timeout = 0
    
    def __init__(self, *args, **kw):
        print("Init")
        super().__init__(*args, **kw)
    
    def setup(self):
        super().setup()
        self.backend = server.backend
        self.headers = {}
        
    def finish(self):
        pass
    
    def handle(self):
        print("handle")
        self.handle_one_request()
        
    # Meat and potatoes
    def do_GET(self):
        for attr in dir(self):
            if not isinstance(attr, type(self.do_GET)) and attr[0] != "_":
                print(attr, getattr(self, attr))
        parsed = url_parse(self.path)
        path = os.path.join(SERVER_DIR, parsed.path)
        if os.path.exists(path):
            self.send_file_reply(200, path)
        elif parsed.path == "/webservice/interface/":
            try:
                self.handle_servercall(parsed)
            except BadServerCall as e:
                self.send_error(e.code, e.msg)
        else:
            self.send_error(503, "ERROR 503: BAD PATH '%s'"%self.path)
            
    # server call handler
    def handle_servercall(self, parsed):
        params = parse_qs(parsed.query)
        call = params.pop("call", None)
        # Check if call is not found or is listed as ""
        if not call:
            call = params.pop("getfile", None)
            if not call:
                raise BadServerCall(405, "No Call Parameter")
        try:
            msg, content_type = self.server.backend.execute(call, params)
        except KeyError as e:
            raise BadServerCall(502, str(e))
        self.send_reply(200, msg, content_type)   

    def send_file_reply(self, code, path):
        
        extension = os.path.splitext(path)[1]
        if extension == ".png":
            content_type = "image/png"
        elif extension == ".css":
            content_type = "text/css"
        elif extension == ".html":
            content_type = "text/html"
        elif extension == ".js":
            content_type = "application/x-javascript"
        elif extension == ".gif":
            content_type = "image/gif"
        elif extension == ".ico":
            content_type = "image/x-icon"
        elif extension == ".min.map":
            content_type = "application/json"
        else:
            self.send_error(501, "Invalid Path Request: '%s'" % path)
            return
        
        with open(path, 'rb') as f:
                body = f.read()
        self.send_reply(code, body, content_type)
        
    def send_reply(self, code, body, content_type):
        if not isinstance(body, bytes):
            body = body.encode("utf-8")
        self.send_response(code)
        for h, v in self.headers.items():
            self.send_header(h, v)
        self.send_header("Content-Length", len(body))
        self.send_header("Content-Type", content_type)
        self.end_headers()
        self.wfile.write(body)
        self.wfile.flush()
        
    def send_error(self, code, msg, content_type=None):
        if isinstance(msg, str):
            msg = msg.encode("utf-8")
        content_type = content_type or "application/json"
        self.send_reply(code, msg, content_type)
    
class Server2(http.server.HTTPServer):
    timeout=0
    def __init__(self, host="localhost", port=12345, backend=None):
        super().__init__((host, port), HelloHandler)
        self.http_connection = None
        self.https_connection = None
        if not backend:
            raise ValueError("Must have backend")
        self.backend = backend
        
    def process_request(self, request, client_address):
        print("process request)")
        self.finish_request(request, client_address)
    
    

In [ ]:
w = Window()
mf = MainValuesFrame(w.tk, json.loads(_mv_json))
backend = ServerBackend(mf)
server = Server2("localhost", 12345, backend)
w.set_server(server)
mf.grid()
w.mainloop()

In [ ]:
import hello.mock.util
import lxml.etree
d = HelloXML(_getconfig_xml.encode()).data['System_Variables']
#d = {"Reply": {"Result": "True", "Message":d}}
#b = hello.mock.util.hello_xml_from_obj(d, "")
b = hello.mock.util.simple_xml_dump(hello.mock.util.hello_tree_from_msg(d, "System_Variables"))
b2 = hello.mock.util.create_hello_xml(d, "System_Variables")
assert b == b2

from io import StringIO
buf = StringIO()
lxml.etree.dump(lxml.etree.XML(b))#, buf)
# import clipboard
# b = "\n".join(l.strip() for l in buf.getvalue().splitlines())
# lxml.etree.dump(lxml.etree.XML(b), pretty_print=True)
# clipboard.copy(b)

In [ ]:
clipboard.copy(getconfig)

In [ ]:
d